# Caso 3

In [ ]:
# ============================================================
# CASO 3 – Escenario realista (clientes_clean / depots_clean / vehicles_clean)
# CVRP con restricción de tipo de vehículo por "zona" (VehicleSizeRestriction)
# ============================================================

import math, pandas as pd
from pyomo.environ import *

# ------------------------------------------------------------
# 0) Cargar datos
# ------------------------------------------------------------
clients  = pd.read_csv("clients_clean.csv")
depots   = pd.read_csv("depots_clean.csv")
vehicles = pd.read_csv("vehicles_clean.csv")

# IDs base (igual que el modelo original)
client_ids = clients["StandardizedID"].tolist()
depot_id   = depots["StandardizedID"].iloc[0]      # usamos solo el primer CD (como en el modelo base)
vehicle_ids= vehicles["StandardizedID"].tolist()

# Demanda y capacidad (capacidad homogénea = primera del archivo, como en el modelo base)
demand = dict(zip(clients["StandardizedID"], clients["Demand"]))
Q      = float(vehicles["Capacity"].iloc[0])

# Coordenadas
coords = {r.StandardizedID:(r.Latitude, r.Longitude) for _, r in clients.iterrows()}
coords[depot_id] = (depots["Latitude"].iloc[0], depots["Longitude"].iloc[0])

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2-lat1); dlmb = math.radians(lon2-lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(math.sqrt(a))

nodes = client_ids + [depot_id]
c = {(i,j):(0.0 if i==j else haversine_km(*coords[i], *coords[j]))
     for i in nodes for j in nodes}

# ------------------------------------------------------------
# 1) Restricción de tipo de vehículo por "zona"
#    Usamos VehicleSizeRestriction (cliente) y VehicleType (vehículo)
#    small van < medium van < light truck
# ------------------------------------------------------------
zone_restriction = dict(zip(clients["StandardizedID"], clients["VehicleSizeRestriction"]))
veh_type         = dict(zip(vehicles["StandardizedID"], vehicles["VehicleType"]))

type_size = {
    "small van"  : 1,
    "medium van" : 2,
    "light truck": 3
}

# allowed[(v,i)] = 1 si el tipo de v es >= restricción del cliente i, 0 si no
allowed = {}
for v in vehicle_ids:
    vt = veh_type[v]
    size_v = type_size[vt]
    for i in client_ids:
        size_req = type_size[zone_restriction[i]]
        allowed[(v, i)] = 1 if size_v >= size_req else 0

# ------------------------------------------------------------
# 2) Modelo en Pyomo (partiendo del modelo base)
# ------------------------------------------------------------
m = ConcreteModel()

m.V    = Set(initialize=vehicle_ids)
m.N    = Set(initialize=client_ids)
m.D    = Set(initialize=[depot_id])
m.Nall = Set(initialize=nodes)

m.Q = Param(initialize=Q)
m.q = Param(m.N, initialize=demand, within=NonNegativeReals)
m.c = Param(m.Nall, m.Nall, initialize=c, within=NonNegativeReals)
m.allow = Param(m.V, m.N, initialize=allowed, within=Binary)

# --- Variables ---
m.x = Var(m.V, m.Nall, m.Nall, within=Binary)   # arco i->j por v
m.y = Var(m.V, within=Binary)                   # usa vehículo v
m.u = Var(m.V, m.Nall, within=NonNegativeReals) # carga restante

# diagonal i->i prohibida
for v in m.V:
    for i in m.Nall:
        m.x[v,i,i].fix(0)

# --- Restricciones CVRP básicas ---
# 1) Cada cliente se visita exactamente una vez (por algún vehículo)
def visit_once(m, i):
    return sum(m.x[v,i,j] for v in m.V for j in m.Nall if j!=i) == 1
m.VisitOnce = Constraint(m.N, rule=visit_once)

# 2) Flujo en clientes (lo que entra = lo que sale, por vehículo)
def flow_cons(m, v, k):
    return sum(m.x[v,i,k] for i in m.Nall if i!=k) - \
           sum(m.x[v,k,j] for j in m.Nall if j!=k) == 0
m.Flow = Constraint(m.V, m.N, rule=flow_cons)

# 3) Depósito - uso de vehículo
def depot_out(m, v):
    d = list(m.D)[0]
    return sum(m.x[v,d,j] for j in m.N) == m.y[v]
def depot_in(m, v):
    d = list(m.D)[0]
    return sum(m.x[v,i,d] for i in m.N) == m.y[v]
m.DepotOut = Constraint(m.V, rule=depot_out)
m.DepotIn  = Constraint(m.V, rule=depot_in)

# 4) Capacidad + anti-subtours ligera (MTZ)
def u_at_depot(m, v):
    d = list(m.D)[0]
    return m.u[v,d] == m.Q
m.UD = Constraint(m.V, rule=u_at_depot)

m.UB = Constraint(m.V, m.Nall, rule=lambda m,v,i: m.u[v,i] <= m.Q)

def load_trans(m, v, i, j):
    if i == j:
        return Constraint.Skip
    if j in m.N:   # sólo clientes consumen
        return m.u[v,j] <= m.u[v,i] - m.q[j] + m.Q*(1 - m.x[v,i,j])
    return Constraint.Skip
m.Load = Constraint(m.V, m.Nall, m.Nall, rule=load_trans)

# --- 5) Restricción de tipo de vehículo por zona (VehicleSizeRestriction) ---
def zone_out(m, v, i):
    if i not in m.N:
        return Constraint.Skip
    return sum(m.x[v,i,j] for j in m.Nall if j!=i) <= m.allow[v,i]

def zone_in(m, v, i):
    if i not in m.N:
        return Constraint.Skip
    return sum(m.x[v,j,i] for j in m.Nall if j!=i) <= m.allow[v,i]

m.ZoneOut = Constraint(m.V, m.Nall, rule=zone_out)
m.ZoneIn  = Constraint(m.V, m.Nall, rule=zone_in)

# --- 6) Costos (misma idea del modelo base, pero simplificado) ---
Cfixed = {v: 50000.0 for v in vehicle_ids}
Cdist  = {v:  2500.0 for v in vehicle_ids}
Ctime  = {v:     0.0 for v in vehicle_ids}
speed_kmph = 30.0
C_fuel = 0.0
C_special = 0.0

m.dv = Expression(m.V, rule=lambda m,v: 
                  sum(m.c[i,j]*m.x[v,i,j] for i in m.Nall for j in m.Nall if i!=j))
m.tv = Expression(m.V, rule=lambda m,v: m.dv[v] / speed_kmph)

m.OBJ = Objective(
    expr = sum(Cfixed[v]*m.y[v] for v in m.V)
         + sum(Cdist[v]*m.dv[v] for v in m.V)
         + sum(Ctime[v]*m.tv[v] for v in m.V)
         + C_fuel + C_special,
    sense = minimize
)

# ------------------------------------------------------------
# 3) Resolver (gap grande, límite de tiempo)
# ------------------------------------------------------------
from pyomo.environ import SolverFactory, value

solver = SolverFactory("highs")  # o "glpk" si no tienes HiGHS
# parámetros para que salga algo rápido (no óptimo)
if solver.name == "highs":
    solver.options["time_limit"]  = 60    # segundos
    solver.options["mip_rel_gap"] = 0.8   # 80% de gap permitido
result = solver.solve(m, tee=False)

print(result.solver.status, result.solver.termination_condition)
print("Valor función objetivo:", value(m.OBJ))

# ------------------------------------------------------------
# 4) Exportar arcos activos a CSV (para revisar)
# ------------------------------------------------------------
import csv

rows = []
for v in m.V:
    for i in m.Nall:
        for j in m.Nall:
            if i != j and m.x[v,i,j].value is not None and m.x[v,i,j].value > 0.5:
                rows.append((v, i, j))

with open("rutas_caso3_clean.csv", "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["VehicleID", "From", "To"])
    for r in rows:
        w.writerow(r)

print("Se guardó rutas_caso3_clean.csv con los arcos usados en la solución.")

aborted maxTimeLimit
Valor función objetivo: 5355421.3645159835
Se guardó rutas_caso3_clean.csv con los arcos usados en la solución.
